TODO

- Handle type mismatch and cast to proper type and not just to string
- metadata columns: (ingestion_timestamp, source_file_path, batch_id)
- [Optional] Dump corrupted or invalid rows (bad casts, missing columns) into quarantine bucket like error_records

In [1]:
# spark.stop()

In [3]:
import os
import pyspark
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, BooleanType
from google.cloud import storage

In [4]:
SPARK_MASTER = "spark://spark-master:7077"

spark = (
    SparkSession
    .builder
    # .master(SPARK_MASTER)
    .appName("process_type_mismatch")
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-2.2.14.jar") # GCS Connector
    .getOrCreate()
)

# Google Cloud Service Account Credentials
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile",os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))

spark

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/30 14:33:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
[
    ('patientid', 'string'),
    ('patientagegroup', 'string'),
    ('patientonsetage', 'integer'),
    ('patientonsetageunit', 'integer'),
    ('patientsex', 'integer'),
    ('patientweight', 'float'),
    ('serious', 'int'),
    ('seriousnessdeath', 'int'),
    ('seriousnesshospitalization', 'int'),
    ('seriousnessdisabling', 'int'),
    ('seriousnesslifethreatening', 'int'),
    ('seriousnessother', 'int'),
    ('receivedate', 'int'),
    ('receiptdate', 'int'),
    ('safetyreportid', 'int')
]

['patientid',
 'patientagegroup',
 'patientonsetage',
 'patientonsetageunit',
 'patientsex',
 'patientweight',
 'serious',
 'seriousnessdeath',
 'seriousnesshospitalization',
 'seriousnessdisabling',
 'seriousnesslifethreatening',
 'seriousnessother',
 'receivedate',
 'receiptdate',
 'safetyreportid']

In [ ]:
def get_year(blob):
    return blob.name.split("/")[3]

def get_filename(blob):
    return blob.name.split('/')[-1].split('.')[0]

def scan_years(blobs): 
    return list({get_year(blob) for blob in blobs})

def process_parquet(bucket, dir, year):
    blobs = list(bucket.list_blobs(prefix=f"{dir}/{year}"))
    for blob in blobs:
        source_blob = f"gs://{bucket.name}/{blob.name}"
        destination_blob = f"gs://{bucket.name}/cleaned/pq/{dir.split('/')[-1]}/{year}/{get_filename(blob)}"

        print(f"Reading file {source_blob}")
        # Read parquet
        df = spark.read.parquet(source_blob)
        
        # Use the transformation package
        # Cast to right types

        # Transform

        # Write to Destination
        df.repartition(4).write.mode("overwrite").parquet(destination_blob)
        print(f"Saved to {destination_blob}")

In [ ]:
# client = storage.Client()
# bucket = client.get_bucket("zoomcamp-454219-ade-pipeline")
# dir = "data/pq/patient"
# year = '2004'

# test = '2004,2005,2006'

# if not test:
#     years = scan_years(list(test =bucket.list_blobs(prefix=dir)))
# else:
#     years = [s for s in test.split(',')]

# for year in years:
#     print(year)
#     # process_parquet(bucket, dir, year)

# # process_parquet(bucket,dir,year)

2004
2005
2006


In [ ]:
client = storage.Client()
bucket = client.get_bucket("zoomcamp-454219-ade-pipeline")
dirs = ["data/pq/patient", "data/pq/drug", "data/pq/reaction"]

# Iterate through each source directory and each year over n parquet files
for dir in dirs:
    # years = scan_years(bucket.list_blobs(prefix=dir))
    years = ['2004']
    for year in years:
        process_parquet(bucket, dir, year)